In [1]:

import numpy as np 
import matplotlib.pyplot as plt

# librería con SVM
from sklearn import svm

# Carga el conjunto de datos y algunas funciones adicionales
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Librería cuántico (Qibo)
import qibo
from qibo import gates
from qibo.models import Circuit

#Uso de backend de numpy en Qibo
qibo.set_backend("numpy")


[Qibo 0.1.12.dev0|INFO|2024-02-15 12:22:07]: Using numpy backend on /CPU:0


In [2]:
X, y = load_iris(return_X_y=True)

# Se selecciona solo la dos primeras clases del conjunto 
# de datos debido a que el clasificador es binario
X = X[:100]
y = y[:100]

# Escalar los datos a distribución normal estándar debido a la 
# codificación datos es periódica
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# Escalar las etiquetas a -1 y 1 debido que es importante para SVM
y_scaled = 2 * (y - 0.5)

# Se divide en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

In [3]:
# kernel cuántico
def distancia_circuit(x,y):
    
    # cuatro cúbits: cuatro características
    n_qubits = len(X_train[0]) 
    n_shots = 8000

    # Create an empty circuit
    circuit = Circuit(n_qubits)

    # Codificación cuántica en puertas RY 
    for pos in range(n_qubits):
        circuit.add(gates.RY(pos, y[pos]))
        circuit.add(gates.RY(pos, x[pos]).dagger())
        circuit.add(gates.M(pos))

    # ejecución del circuito n veces (n_shots)
    result = circuit.execute(nshots=n_shots)
    print('result',result.probabilities())
    # Devuelve las probabilidades analíticas
    return result.probabilities()
       
# Se devuelve el probabilidad del base computacional "0000"
kernel = lambda x1, x2: distancia_circuit(x1, x2)[0]

In [4]:
x = np.array([np.pi/2,np.pi/2,np.pi/2,np.pi/2])
kernel(x,x)

result [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


1.0

In [5]:
def kernel_matrix(A, B):
    return np.array([[kernel(a, b) for b in B] for a in A])

In [6]:
# Inicialización
qsvm = svm.SVC(kernel=kernel_matrix)

print("Entrenando...")
qsvm = qsvm.fit(X_train, y_train)

Entrenando...
result [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
result [0.22173894 0.11417503 0.1328347  0.06839758 0.1810265  0.09321189
 0.10844555 0.05583942 0.00552953 0.0028472  0.00331251 0.00170564
 0.00451428 0.00232444 0.00270432 0.00139247]
result [0.33978115 0.07711433 0.06631855 0.01505119 0.27739554 0.06295574
 0.05414212 0.01228771 0.03564841 0.00809051 0.00695786 0.00157911
 0.02910317 0.00660505 0.00568036 0.00128918]
result [0.03660827 0.01884987 0.02527878 0.01301623 0.35386682 0.18220866
 0.24435247 0.12581891 0.         0.         0.         0.
 0.         0.         0.         0.        ]
result [4.78774194e-06 2.46524400e-06 1.83778436e-06 9.46288860e-07
 3.47457519e-01 1.78908465e-01 1.33372266e-01 6.86743737e-02
 1.78501066e-06 9.19115291e-07 6.85179926e-07 3.52804250e-07
 1.29542357e-01 6.67023249e-02 4.97250938e-02 2.56038213e-02]
result [0.11151342 0.08283221 0.06680312 0.04962138 0.21324369 0.15839751
 0.12774555 0.09488945 0.01169952 0.00869041 0.007

In [7]:
#test
print("Precisión con test...")

predictions = qsvm.predict(X_test)
accuracy_score(predictions, y_test)

Precisión con test...
result [3.84653042e-02 1.98060718e-02 2.30429866e-02 1.18650055e-02
 3.71817523e-01 1.91451613e-01 2.22740633e-01 1.14690812e-01
 2.36859198e-04 1.21960566e-04 1.41892634e-04 7.30615744e-05
 2.28955423e-03 1.17890854e-03 1.37157807e-03 7.06235767e-04]
result [7.08346865e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.37467932e-01 7.70371978e-34 0.00000000e+00 0.00000000e+00
 4.05807969e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.36044054e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
result [0.03740375 0.12309668 0.12767263 0.4201738  0.01253933 0.04126723
 0.04280128 0.14086009 0.00214284 0.00705214 0.00731429 0.02407152
 0.00071837 0.00236418 0.00245206 0.0080698 ]
result [9.92685963e-01 0.00000000e+00 1.19398602e-03 3.00926554e-36
 3.07778600e-33 9.49556775e-66 3.70190936e-36 0.00000000e+00
 6.11269832e-03 0.00000000e+00 7.35225098e-06 0.00000000e+00
 4.80904063e-35 1.48368246e-67 5.78423338e-38 0.00000000e+00]
result [7.10993823e-01 3.08148791e-

1.0

In [8]:
modelclasico = svm.SVC(kernel="linear")
modelclasico.fit(X_train, y_train)

print("Precisión con test...")

predictions = modelclasico.predict(X_test)
accuracy_score(predictions, y_test)


Precisión con test...


1.0